### Word Group Prepare For Youtube Tool

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Turkish
Lang Id = 2
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group Prepare For Youtube Tool"

Path(path).mkdir(parents=True, exist_ok=True)

#### Read Data

In [6]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [7]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(700)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,yok çok değil o kadar da değil,4267,4268,yok çok değil o kadar da değil,IGJVw1HjIHc,https://www.youtube.com/watch?v=IGJVw1HjIHc&t=...
1,yok yok yok sen onu değil bunu,5166,5168,yok yok yok sen onu değil bunu al,t4G3MB0ULX4,https://www.youtube.com/watch?v=t4G3MB0ULX4&t=...
2,ama bu o kadar şey değil evet,394,397,ama bu o kadar şey değil evet,U4CqkDvAE_Q,https://www.youtube.com/watch?v=U4CqkDvAE_Q&t=...
3,da var yok yok daha çok var,2037,2039,yarına da var yok yok daha çok var,zXR_pFRSE7s,https://www.youtube.com/watch?v=zXR_pFRSE7s&t=...
4,var gibi de ama yok gibi de,64,68,var gibi de ama yok gibi de,ghAr1nU391c,https://www.youtube.com/watch?v=ghAr1nU391c&t=64s
...,...,...,...,...,...,...
695,bana da yok,3920,3922,bana da yok,EqVZ1Bet5U0,https://www.youtube.com/watch?v=EqVZ1Bet5U0&t=...
696,daha var bu,6255,6257,daha var bu,SnbDtuHGdqE,https://www.youtube.com/watch?v=SnbDtuHGdqE&t=...
697,daha var mı,2477,2480,daha var mı,rQhGmJxi_s4,https://www.youtube.com/watch?v=rQhGmJxi_s4&t=...
698,ben mi evet,2026,2029,ben mi evet,6Pr093ZQF5A,https://www.youtube.com/watch?v=6Pr093ZQF5A&t=...


In [8]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(700)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
1,bilmiyorum sen ne demek istiyorsun ben de onu ...,721,726,bilmiyorum sen ne demek istiyorsun ben de onu ...,ZdeFof81xKs,https://www.youtube.com/watch?v=ZdeFof81xKs&t=...
2,var da bu bir sorun olur mu benim için ben bun...,7920,7926,var da bu bir sorun olur mu benim için ben bun...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
3,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...
4,gerçekten çok teşekkür ederim ya bu çok büyük ...,2257,2262,gerçekten çok teşekkür ederim ya bu çok büyük ...,a8Dov7B7alQ,https://www.youtube.com/watch?v=a8Dov7B7alQ&t=...
...,...,...,...,...,...,...
695,böyle şeyler çünkü bilmiyorum bence,400,407,böyle şeyler çünkü bilmiyorum bence normal,9NguZ1ukR5E,https://www.youtube.com/watch?v=9NguZ1ukR5E&t=...
696,tabii ne kadar doğru bilmiyorum ama,1173,1175,tabii ne kadar doğru bilmiyorum ama,6TOnCfxXzyU,https://www.youtube.com/watch?v=6TOnCfxXzyU&t=...
697,ben çok teşekkür ederim sen de öyle,70,74,ben çok teşekkür ederim sen de öyle,5JLIqCc8zpA,https://www.youtube.com/watch?v=5JLIqCc8zpA&t=70s
698,çok teşekkürler ben teşekkür ederim,1786,1789,okey çok teşekkürler ben teşekkür ederim,Ssg7vpw8Zb4,https://www.youtube.com/watch?v=Ssg7vpw8Zb4&t=...


In [9]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(700)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...
1,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,https://www.youtube.com/watch?v=N4YDaU51lvI&t=...
2,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,https://www.youtube.com/watch?v=cUAQrHbbpZ8&t=...
3,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...
4,sıcak mı evet böyle işte tamam bir şey yok ora...,603,612,sıcak mı evet böyle işte tamam bir şey yok ora...,Mh3bgDLKt2o,https://www.youtube.com/watch?v=Mh3bgDLKt2o&t=...
...,...,...,...,...,...,...
695,sen oradan çık diye neler yaptım sen biliyor m...,4649,4651,sen oradan çık diye neler yaptım sen biliyor m...,R4o2jakVqNg,https://www.youtube.com/watch?v=R4o2jakVqNg&t=...
696,daha kolay ve hızlı ilk olarak bunu kabul ediy...,57,61,daha kolay ve hızlı ilk olarak bunu kabul ediy...,nYNwFO8hA2g,https://www.youtube.com/watch?v=nYNwFO8hA2g&t=57s
697,onu verir misin yapar mısın diye böyle yap diy...,1380,1384,i şte onu verir misin yapar mısın diye böyle y...,Yr2ZLfWT0i8,https://www.youtube.com/watch?v=Yr2ZLfWT0i8&t=...
698,gelip gerçekten iyi olup olmadığını görmek ist...,3559,3563,gelip gerçekten iyi olup olmadığını görmek ist...,bLSG6R3LgeY,https://www.youtube.com/watch?v=bLSG6R3LgeY&t=...


In [10]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(700)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,şimdi ben seni tabii hem kişisel olarak hem de...,380,396,şimdi ben seni tabii hem kişisel olarak hem de...,MQ_nenSkjbo,https://www.youtube.com/watch?v=MQ_nenSkjbo&t=...
1,ama ben kimseye durup da spor yapmak isteyen i...,399,407,ama ben kimseye durup da spor yapmak isteyen i...,dhV8ToP5VTQ,https://www.youtube.com/watch?v=dhV8ToP5VTQ&t=...
2,bana bak hadi gidip bana da bir şeyler daha al...,3243,3249,bana bak hadi gidip bana da bir şeyler daha al...,Ws5feMfuQ7I,https://www.youtube.com/watch?v=Ws5feMfuQ7I&t=...
3,yani tabii inanıyorum aşkın çok büyük bir güç ...,1893,1903,yani tabii inanıyorum aşkın çok büyük bir güç ...,1UOatSFd_hE,https://www.youtube.com/watch?v=1UOatSFd_hE&t=...
4,hani benim için özel bir gün olacaktı ya bugün...,2774,2784,hani benim için özel bir gün olacaktı ya bugün...,HjuzPWNQeP4,https://www.youtube.com/watch?v=HjuzPWNQeP4&t=...
...,...,...,...,...,...,...
695,ya nereden geliyorsun arkadaş sen ya benim ora...,600,610,ya nereden geliyorsun arkadaş sen ya benim ora...,v_ZhT1JJe4g,https://www.youtube.com/watch?v=v_ZhT1JJe4g&t=...
696,hemen dediğimi yapın ali benimle pazarlık edeb...,2191,2196,hemen dediğimi yapın ali benimle pazarlık edeb...,LJiP_ouvA9k,https://www.youtube.com/watch?v=LJiP_ouvA9k&t=...
697,sistem arabanın beyni kendi kendine onu kontro...,533,539,taktıysa nız sistem arabanın beyni kendi kendi...,rls5dLqv3TY,https://www.youtube.com/watch?v=rls5dLqv3TY&t=...
698,en soğuk olanı buydu oldukça sıcak durdu üstün...,909,919,kontürünü en soğuk olanı buydu oldukça sıcak d...,FzTiUa3nlRo,https://www.youtube.com/watch?v=FzTiUa3nlRo&t=...


In [11]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(700)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,on tane geliyor haftada üç dört kere şeye çıkı...,679,690,on tane geliyor haftada üç dört kere şeye çıkı...,LPrMSJHDDcA,https://www.youtube.com/watch?v=LPrMSJHDDcA&t=...
1,okulun ilk günü ne yapacağımızı bilmiyorduk bi...,695,705,okulun ilk günü ne yapacağımızı bilmiyorduk bi...,FIcnpr3UtaQ,https://www.youtube.com/watch?v=FIcnpr3UtaQ&t=...
2,şimdi ben seni tabii hem kişisel olarak hem de...,380,396,şimdi ben seni tabii hem kişisel olarak hem de...,MQ_nenSkjbo,https://www.youtube.com/watch?v=MQ_nenSkjbo&t=...
3,o yüzden aslında hep sosyal medya ile bir şeki...,109,119,o yüzden aslında hep sosyal medya ile bir şeki...,ywf4HpQKuf0,https://www.youtube.com/watch?v=ywf4HpQKuf0&t=...
4,ve bu gözle gelecek olan ikinci karakteri ki ö...,362,371,ve bu gözle gelecek olan ikinci karakteri ki ö...,nB0Q5v1_kUs,https://www.youtube.com/watch?v=nB0Q5v1_kUs&t=...
...,...,...,...,...,...,...
695,her zamanki gibi nereden aldığımı zaten biliyo...,30,35,her zamanki gibi nereden aldığımı zaten biliyo...,n5b8T70avq8,https://www.youtube.com/watch?v=n5b8T70avq8&t=30s
696,söylüyorlar öyle merak ettik geldik bakalım sö...,1007,1013,söylüyorlar öyle merak ettik geldik bakalım sö...,hLM07VHy0xI,https://www.youtube.com/watch?v=hLM07VHy0xI&t=...
697,yani baş parmak öne gelecek şekilde arkadaşlar...,654,660,yani baş parmak öne gelecek şekilde arkadaşlar...,xhdaMIJbk_Y,https://www.youtube.com/watch?v=xhdaMIJbk_Y&t=...
698,diğer tarafa dönüyorum ağrı hissettiğim an dah...,583,590,diğer tarafa dönüyorum ağrı hissettiğim an dah...,A9OgGY5SCvE,https://www.youtube.com/watch?v=A9OgGY5SCvE&t=...


In [12]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(700)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,on tane geliyor haftada üç dört kere şeye çıkı...,679,690,on tane geliyor haftada üç dört kere şeye çıkı...,LPrMSJHDDcA,https://www.youtube.com/watch?v=LPrMSJHDDcA&t=...
1,ancak herhangi bir sorusu varsa ya da belki bi...,682,691,ancak herhangi bir sorusu varsa ya da belki bi...,J0hq13epy2k,https://www.youtube.com/watch?v=J0hq13epy2k&t=...
2,oradaki sadece şu çok önemliydi dört kere geld...,3834,3843,oradaki sadece şu çok önemliydi dört kere geld...,LOQOS5IJtlM,https://www.youtube.com/watch?v=LOQOS5IJtlM&t=...
3,sen hayattan ne istiyorsan onun için çalışmalı...,1342,1377,sen hayattan ne istiyorsan onun için çalışmalı...,luv56Zk_gkY,https://www.youtube.com/watch?v=luv56Zk_gkY&t=...
4,herhalde geldiğim en yoksul ülke burası oldu y...,1074,1115,herhalde geldiğim en yoksul ülke burası oldu y...,9jCA8ZQD2IY,https://www.youtube.com/watch?v=9jCA8ZQD2IY&t=...
...,...,...,...,...,...,...
695,bunu önemli yapabilirsiniz kampanya adını sorm...,896,902,bunu önemli yapabilirsiniz kampanya adını sorm...,UctkmnbuBIA,https://www.youtube.com/watch?v=UctkmnbuBIA&t=...
696,böyle yaptığın zaman hepsi aynı oranda düzgünc...,61,70,böyle yaptığın zaman hepsi aynı oranda düzgünc...,SNGC_rs6F24,https://www.youtube.com/watch?v=SNGC_rs6F24&t=61s
697,kısmına giriyoruz aslında burada dört tane dör...,25,33,kısmına giriyoruz aslında burada dört tane dör...,TfwT8uTffO8,https://www.youtube.com/watch?v=TfwT8uTffO8&t=25s
698,arkadaşlar nasılsınız beni soracak olursanız h...,2,48,arkadaşlar nasılsınız beni soracak olursanız h...,uLM3hC_A9B0,https://www.youtube.com/watch?v=uLM3hC_A9B0&t=2s


In [13]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(700)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,baktığınızda aynı fikirde misiniz çoğunlukla h...,410,425,baktığınızda aynı fikirde misiniz çoğunlukla h...,CULrx3yerE8,https://www.youtube.com/watch?v=CULrx3yerE8&t=...
1,üzülme yarın beraber ararız belki de o çiçeği ...,608,626,üzülme yarın beraber ararız belki de o çiçeği ...,69Z_fImC1v0,https://www.youtube.com/watch?v=69Z_fImC1v0&t=...
2,on tane geliyor haftada üç dört kere şeye çıkı...,679,690,on tane geliyor haftada üç dört kere şeye çıkı...,LPrMSJHDDcA,https://www.youtube.com/watch?v=LPrMSJHDDcA&t=...
3,yeni yılda eve şans gelmesi için gece yarısınd...,238,251,yeni yılda eve şans gelmesi için gece yarısınd...,aAa2PRgjkcg,https://www.youtube.com/watch?v=aAa2PRgjkcg&t=...
4,bir sürü kızla çıkmış bir çocuk kızları düşünü...,587,595,bir sürü kızla çıkmış bir çocuk kızları düşünü...,bwEEzF905Gk,https://www.youtube.com/watch?v=bwEEzF905Gk&t=...
...,...,...,...,...,...,...
695,bunu biliyor musunuz hiç benim bildiğim kadarı...,759,766,bunu biliyor musunuz hiç benim bildiğim kadarı...,5jZCJvqswAU,https://www.youtube.com/watch?v=5jZCJvqswAU&t=...
696,yatak odası göreceksiniz şimdi salonu gezelim ...,658,669,yatak odası göreceksiniz şimdi salonu gezelim ...,CAGp6IVFnf0,https://www.youtube.com/watch?v=CAGp6IVFnf0&t=...
697,bunun gibi tonlarca yeni şeyleri sizlerle sist...,1644,1650,bunun gibi tonlarca yeni şeyleri sizlerle sist...,bRIjl3yF01U,https://www.youtube.com/watch?v=bRIjl3yF01U&t=...
698,konuya girmek istiyorum birkaç tane seri yapmı...,38,46,konuya girmek istiyorum birkaç tane seri yapmı...,6RIWaTqZ5Tg,https://www.youtube.com/watch?v=6RIWaTqZ5Tg&t=38s


#### End Time Reassign

In [14]:
df_28["end_time"] = df_28["start_time"] + 1
df_200["end_time"] = df_200["start_time"] + 1
df_1000["end_time"] = df_1000["start_time"] + 1
df_5000["end_time"] = df_5000["start_time"] + 1
df_10000["end_time"] = df_10000["start_time"] + 1
df_20000["end_time"] = df_20000["start_time"] + 1
df_40000["end_time"] = df_40000["start_time"] + 1

In [15]:
out_path = "/home/kurubal/Downloads"

In [16]:
df_28.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_200.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_1000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_5000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_10000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_20000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_40000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)

In [ ]:
# End Time Reassign files result check on youtube tool and change file with youtube tool file result as same name (old file should remove) 

#### End Time Reassign Convert Orjinal (From Youtube Tool Result)

In [16]:
df_28_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28_video_list = list(set(df_28_youtube_tool["video_id"]))

FileNotFoundError: [Errno 2] No such file or directory: '/home/kurubal/Downloads/Turkish_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx'

In [ ]:
df_200_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200_video_list = list(set(df_200_youtube_tool["video_id"]))

In [ ]:
df_1000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000_video_list = list(set(df_1000_youtube_tool["video_id"]))

In [ ]:
df_5000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000_video_list = list(set(df_5000_youtube_tool["video_id"]))

In [ ]:
df_10000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000_video_list = list(set(df_10000_youtube_tool["video_id"]))

In [17]:
df_20000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000_video_list = list(set(df_20000_youtube_tool["video_id"]))

In [18]:
df_40000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000_video_list = list(set(df_40000_youtube_tool["video_id"]))

##### Video List In video_id 

In [19]:
df_28_new = df_28[df_28["video_id"].isin(df_28_video_list)]
#df_28_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv", index=False)
df_28_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_28_new

NameError: name 'df_28_video_list' is not defined

In [ ]:
df_200_new = df_200[df_200["video_id"].isin(df_200_video_list)]
#df_200_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv", index=False)
df_200_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_200_new

In [ ]:
df_1000_new = df_1000[df_1000["video_id"].isin(df_1000_video_list)]
#df_1000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv", index=False)
df_1000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_1000_new

In [ ]:
df_5000_new = df_5000[df_5000["video_id"].isin(df_5000_video_list)]
#df_5000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv", index=False)
df_5000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_5000_new

In [ ]:
df_10000_new = df_10000[df_10000["video_id"].isin(df_10000_video_list)]
#df_10000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv", index=False)
df_10000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_10000_new

In [26]:
df_20000_new = df_20000[df_20000["video_id"].isin(df_20000_video_list)]
#df_20000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv", index=False)
df_20000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_20000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,on tane geliyor haftada üç dört kere şeye çıkı...,679,690,on tane geliyor haftada üç dört kere şeye çıkı...,LPrMSJHDDcA,https://www.youtube.com/watch?v=LPrMSJHDDcA&t=...
1,ancak herhangi bir sorusu varsa ya da belki bi...,682,691,ancak herhangi bir sorusu varsa ya da belki bi...,J0hq13epy2k,https://www.youtube.com/watch?v=J0hq13epy2k&t=...
2,oradaki sadece şu çok önemliydi dört kere geld...,3834,3843,oradaki sadece şu çok önemliydi dört kere geld...,LOQOS5IJtlM,https://www.youtube.com/watch?v=LOQOS5IJtlM&t=...
3,sen hayattan ne istiyorsan onun için çalışmalı...,1342,1377,sen hayattan ne istiyorsan onun için çalışmalı...,luv56Zk_gkY,https://www.youtube.com/watch?v=luv56Zk_gkY&t=...
4,herhalde geldiğim en yoksul ülke burası oldu y...,1074,1115,herhalde geldiğim en yoksul ülke burası oldu y...,9jCA8ZQD2IY,https://www.youtube.com/watch?v=9jCA8ZQD2IY&t=...
...,...,...,...,...,...,...
695,bunu önemli yapabilirsiniz kampanya adını sorm...,896,902,bunu önemli yapabilirsiniz kampanya adını sorm...,UctkmnbuBIA,https://www.youtube.com/watch?v=UctkmnbuBIA&t=...
696,böyle yaptığın zaman hepsi aynı oranda düzgünc...,61,70,böyle yaptığın zaman hepsi aynı oranda düzgünc...,SNGC_rs6F24,https://www.youtube.com/watch?v=SNGC_rs6F24&t=61s
697,kısmına giriyoruz aslında burada dört tane dör...,25,33,kısmına giriyoruz aslında burada dört tane dör...,TfwT8uTffO8,https://www.youtube.com/watch?v=TfwT8uTffO8&t=25s
698,arkadaşlar nasılsınız beni soracak olursanız h...,2,48,arkadaşlar nasılsınız beni soracak olursanız h...,uLM3hC_A9B0,https://www.youtube.com/watch?v=uLM3hC_A9B0&t=2s


In [27]:
df_40000_new = df_40000[df_40000["video_id"].isin(df_40000_video_list)]
#df_40000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv", index=False)
df_40000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)
df_40000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,baktığınızda aynı fikirde misiniz çoğunlukla h...,410,425,baktığınızda aynı fikirde misiniz çoğunlukla h...,CULrx3yerE8,https://www.youtube.com/watch?v=CULrx3yerE8&t=...
2,on tane geliyor haftada üç dört kere şeye çıkı...,679,690,on tane geliyor haftada üç dört kere şeye çıkı...,LPrMSJHDDcA,https://www.youtube.com/watch?v=LPrMSJHDDcA&t=...
3,yeni yılda eve şans gelmesi için gece yarısınd...,238,251,yeni yılda eve şans gelmesi için gece yarısınd...,aAa2PRgjkcg,https://www.youtube.com/watch?v=aAa2PRgjkcg&t=...
4,bir sürü kızla çıkmış bir çocuk kızları düşünü...,587,595,bir sürü kızla çıkmış bir çocuk kızları düşünü...,bwEEzF905Gk,https://www.youtube.com/watch?v=bwEEzF905Gk&t=...
5,cumartesi bildiğin iki saat aralıksız dört far...,333,345,cumartesi bildiğin iki saat aralıksız dört far...,LPrMSJHDDcA,https://www.youtube.com/watch?v=LPrMSJHDDcA&t=...
...,...,...,...,...,...,...
695,bunu biliyor musunuz hiç benim bildiğim kadarı...,759,766,bunu biliyor musunuz hiç benim bildiğim kadarı...,5jZCJvqswAU,https://www.youtube.com/watch?v=5jZCJvqswAU&t=...
696,yatak odası göreceksiniz şimdi salonu gezelim ...,658,669,yatak odası göreceksiniz şimdi salonu gezelim ...,CAGp6IVFnf0,https://www.youtube.com/watch?v=CAGp6IVFnf0&t=...
697,bunun gibi tonlarca yeni şeyleri sizlerle sist...,1644,1650,bunun gibi tonlarca yeni şeyleri sizlerle sist...,bRIjl3yF01U,https://www.youtube.com/watch?v=bRIjl3yF01U&t=...
698,konuya girmek istiyorum birkaç tane seri yapmı...,38,46,konuya girmek istiyorum birkaç tane seri yapmı...,6RIWaTqZ5Tg,https://www.youtube.com/watch?v=6RIWaTqZ5Tg&t=38s


#### Copy Move And Delete

In [29]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_*_Word{file_ext}.*")
output_file2

['Turkish_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [30]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [31]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass